In [ ]:
# Dataset-1: AMAZON FASHION 	reviews (883,636 reviews)	metadata (186,637 products)
# Dataset-2: All Beauty 	reviews (371,345 reviews)	metadata (32,992 products)
# Dataset-3: Appliances 	reviews (602,777 reviews)	metadata (30,459 products)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
#nltk.download('vader_lexicon')
import nltk.corpus
import os
import sys
import json

from collections import Counter
"""
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')
!ls "/content/drive/MyDrive/Review"

data = [json.loads(line) for line in open('/content/drive/MyDrive/Review/Magazine_Subscriptions.json', 'r')]
print(len(data))
for item in data:
  data['overall'].value_count() 
"""

data = [json.loads(line) for line in open('Appliances.json', 'r')]
no_data=len(data)

i=0
overall=[None]*no_data
reviewText=[None]*no_data
reviewScore=0

for item in data:
    if "reviewText" in item:
        overall[i]=item['overall']
        reviewText[i]=item['reviewText']
        reviewScore+=overall[i]
        i=i+1
    else:
        no_data-=1
        
print(no_data)    


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sample_review = reviewText[:no_data]

sentiment = SentimentIntensityAnalyzer()

f=open('sentiment.txt', 'w')
count=0
avg_rating=0
comp_rating=0
for sentences in sample_review:
    sentences
    ss = sentiment.polarity_scores(sentences)
    
    #to get compound, negetive, neutral, positive score
    for k in sorted(ss):
        compound_score =ss['compound']
        #print(f"ss: {compound_score}")
    #f.write(f"ss[{count}]:{(compound_score+1)/2.0*5.0}\n")
    
    avg_rating+=(compound_score+1)/2.0*5.0
    comp_rating+=(overall[count]+compound_score/5.0)*(5/6)
    #f.write(sentences)
    #f.write("\n")
    count+=1
    
     
    """    
        print('{0}: {1}, '.format(k, ss[k]))
    print(sentences)   
   
    """
f.close()
review_NLP = avg_rating/no_data
review_Score= reviewScore/no_data
comp_rating=comp_rating/no_data
print(f"Review analysis score: {review_Score}")
print(f"Review analysis score using NLP: {review_NLP}")
print(f"Review analysis score tuning Using NLP: {comp_rating}")

   

In [ ]:
##### Forecasting with ARIMA#############

import json
import pandas as pd
from pmdarima.arima import ARIMA
import matplotlib.pyplot as plt

# Load the data from JSON
data = [json.loads(line) for line in open('Appliances.json', 'r')]

# Convert JSON data to a pandas DataFrame
df = pd.DataFrame(data)

# Preprocess the data if needed (e.g., convert date/time columns, handle missing values)
##############
df.rename({'Time': 'unixReviewTime', 'Sales': 'reviewerID'}, axis=1, inplace=True)
df = df[['unixReviewTime', 'reviewerID']].groupby(['unixReviewTime']).agg(['count'])
df = df.reset_index()
df['unixReviewTime'] = df['unixReviewTime'].astype(int)
############

# Set the column names for date and target variable
date_column = 'unixReviewTime'
target_column = 'reviewerID'




# Convert the date column to datetime type
df[date_column] = pd.to_datetime(df[date_column])

# Set the date column as the DataFrame index
df.set_index(date_column, inplace=True)

# Split the data into train and test sets
train_size = int(0.8 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Fit the ARIMA model
order = (1, 1, 1)  # Specify the order of the ARIMA model (p, d, q)
model = ARIMA(order=order)
model_fit = model.fit(train_data[target_column].values)

# Forecast future values with 30% confidence interval
forecast_values, conf_int = model_fit.predict(n_periods=len(test_data), return_conf_int=True, alpha=0.3)

# Create a DataFrame for the forecast values
forecast_df = pd.DataFrame(forecast_values, columns=['forecast'])
forecast_df.index = test_data.index

# Combine the forecast DataFrame with the original test_data DataFrame
forecast_data = pd.concat([test_data, forecast_df], axis=1)

# Plot the train data as a line graph
plt.figure(figsize=(8, 5))
plt.plot(train_data.index, train_data[target_column], label='Train Data', linewidth=2)

# Plot the forecasted values with 30% confidence interval
plt.plot(forecast_data.index, forecast_data['forecast'], label='Forecast')
plt.fill_between(forecast_data.index, conf_int[:, 0], conf_int[:, 1], alpha=0.3)


plt.xlabel('Unix Time')
plt.ylabel('Sales')
plt.title('ARIMA Forecast (with 30% confidence value)')
plt.legend()
plt.show()



In [ ]:
##### ARIMA MAPE Value #############
actual_values = test_data[target_column].values
mape = (abs(actual_values - forecast_values) / actual_values).mean() * 100
print(f"MAPE using ARIMA: {mape:.3f}")

##### ARIMA MAPE Value using customer's review score #############
actual_values = (test_data[target_column].values)
mape = (abs(actual_values - (forecast_values * (review_Score/5) ))/ actual_values).mean() * 100
print(f"MAPE using ARIMA with review score: {mape:.3f}")

##### ARIMA MAPE Value using customer's review text #############
actual_values = (test_data[target_column].values)
mape = (abs(actual_values - (forecast_values * (review_NLP/5) )) / actual_values).mean() * 100
print(f"MAPE using ARIMA with review text: {mape:.3f}")

##### ARIMA MAPE Value using customer's review Score tuning #############
actual_values = (test_data[target_column].values)
mape = (abs(actual_values - (forecast_values * (comp_rating/5) )) / actual_values).mean() * 100
print(f"MAPE using ARIMA with review score tuning: {mape:.3f}")

In [ ]:
# Calculate MSLE (Mean Squared Logarithmic Error)
msle = np.mean(np.log1p((actual_values - forecast_values) ** 2))
print(f"MSLE using ARIMA: {msle:.3f}")

##### ARIMA MSLE Value using customer's review score #############
msle = np.mean(np.log1p((actual_values - forecast_values * (review_Score/5)) ** 2))
print(f"MSLE using ARIMA with review score: {msle:.3f}")

##### ARIMA MSLE Value using customer's review text #############
msle = np.mean(np.log1p((actual_values - forecast_values * (review_NLP/5)) ** 2))
print(f"MSLE using ARIMA with review text: {msle:.3f}")

##### ARIMA MSLE Value using customer's review Score tuning #############
msle = np.mean(np.log1p((actual_values - forecast_values * (comp_rating/5)) ** 2))
print(f"MSLE using ARIMA with review score tuning: {msle:.3f}")



In [ ]:
####### Fit in SARIMA ###########
import json
import pandas as pd
from pmdarima.arima import ARIMA
import matplotlib.pyplot as plt

# Load the data from JSON
data = [json.loads(line) for line in open('Appliances.json', 'r')]

# Convert JSON data to a pandas DataFrame
df = pd.DataFrame(data)

# Preprocess the data if needed (e.g., convert date/time columns, handle missing values)
##############
df.rename({'review_time': 'unixReviewTime', 'product_id': 'reviewerID'}, axis=1, inplace=True)
        
df = df[['unixReviewTime', 'reviewerID']].groupby(['unixReviewTime']).agg(['count'])
df = df.reset_index()
df['unixReviewTime'] = df['unixReviewTime'].astype(int)

############

# Set the column names for date and target variable
date_column = 'unixReviewTime'
target_column = 'reviewerID'

# Convert the date column to datetime type
df[date_column] = pd.to_datetime(df[date_column])

# Set the date column as the DataFrame index
df.set_index(date_column, inplace=True)

# Split the data into train and test sets
train_size = int(0.8 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Fit the SARIMA model
order = (1, 1, 1)  # Specify the order of the ARIMA model (p, d, q)
seasonal_order = (1, 0, 1, 12)  # Specify the seasonal order of the SARIMA model (P, D, Q, S)

SARIMA = ARIMA(order=order, seasonal_order=seasonal_order)
model_fit = SARIMA.fit(train_data[target_column].values)

# Forecast future values for the test data
forecast_values, conf_int = model_fit.predict(n_periods=len(test_data), return_conf_int=True, alpha=0.3)

# Create a DataFrame for the forecast values
forecast_df = pd.DataFrame(forecast_values, columns=['forecast'])
forecast_df.index = test_data.index

# Combine the forecast DataFrame with the original test_data DataFrame
forecast_data = pd.concat([test_data, forecast_df], axis=1)

# Plot the forecast and confidence interval for the test data
plt.figure(figsize=(8, 5))
plt.plot(train_data.index, train_data[target_column], label='Train Data')
plt.plot(test_data.index, forecast_data['forecast'], label='Forecast')
plt.fill_between(test_data.index, conf_int[:, 0], conf_int[:, 1], alpha=0.3, color='gray', label='Confidence Interval')
plt.xlabel('Unix Time')
plt.ylabel('Sales')
plt.title('SARIMA Forecast (with 30% confidence value)')
plt.legend()
plt.show()


In [ ]:
##### SARIMA MAPE Value #############
actual_values = test_data[target_column].values
mape = (abs(actual_values - forecast_values) / actual_values).mean() * 100
print(f"MAPE using SARIMA: {mape:.3f}")

##### SARIMA MAPE Value using customer's review score #############
actual_values = (test_data[target_column].values)
mape = (abs(actual_values - (forecast_values * (review_Score/5) ))/ actual_values).mean() * 100
print(f"MAPE using SARIMA with review score: {mape:.3f}")

##### SARIMA MAPE Value using customer's review text #############
actual_values = (test_data[target_column].values)
mape = (abs(actual_values - (forecast_values * (review_NLP/5) )) / actual_values).mean() * 100
print(f"MAPE using SARIMA with review text: {mape:.3f}")

##### SARIMA MAPE Value using customer's review score tuning #############
actual_values = (test_data[target_column].values)
mape = (abs(actual_values - (forecast_values * (comp_rating/5) )) / actual_values).mean() * 100
print(f"MAPE using SARIMA with review score tuning: {mape:.3f}")


In [ ]:
# Calculate MSLE (Mean Squared Logarithmic Error)
msle = np.mean(np.log1p((actual_values - forecast_values) ** 2))
print(f"MSLE using SARIMA: {msle:.3f}")

##### SARIMA MSLE Value using customer's review score #############
msle = np.mean(np.log1p((actual_values - forecast_values * (review_Score/5)) ** 2))
print(f"MSLE using SARIMA with review score: {msle:.3f}")

##### SARIMA MSLE Value using customer's review text #############
msle = np.mean(np.log1p((actual_values - forecast_values * (review_NLP/5)) ** 2))
print(f"MSLE using SARIMA with review text: {msle:.3f}")

##### SARIMA MSLE Value using customer's review score tuning #############
msle = np.mean(np.log1p((actual_values - forecast_values * (comp_rating/5)) ** 2))
print(f"MSLE using SARIMA with review score tuning: {msle:.3f}")

In [ ]:
#### LSTM model ########

import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load the data from JSON
data = [json.loads(line) for line in open('Appliances.json', 'r')]

# Convert JSON data to a pandas DataFrame
df = pd.DataFrame(data)


# Preprocess the data if needed (e.g., convert date/time columns, handle missing values)
##############
df.rename({'review_time': 'unixReviewTime', 'product_id': 'reviewerID'}, axis=1, inplace=True)
        
df = df[['unixReviewTime', 'reviewerID']].groupby(['unixReviewTime']).agg(['count'])
df = df.reset_index()
df['unixReviewTime'] = df['unixReviewTime'].astype(int)

############

# Set the column names for date and target variable
date_column = 'unixReviewTime'
target_column = 'reviewerID'

# Convert the date column to datetime type
df[date_column] = pd.to_datetime(df[date_column])

# Set the date column as the DataFrame index
df.set_index(date_column, inplace=True)

# Extract the target variable data
target_data = df[target_column].values.reshape(-1, 1)

# Scale the target variable to the range of 0 to 1
scaler = MinMaxScaler()
scaled_target_data = scaler.fit_transform(target_data)

# Split the data into train and test sets
train_size = int(0.8 * len(df))
train_data = scaled_target_data[:train_size]
test_data = scaled_target_data[train_size:]

# Define the window size for input sequence
window_size = 10

# Prepare the training data
X_train = []
y_train = []
for i in range(window_size, len(train_data)):
    X_train.append(train_data[i - window_size:i, 0])
    y_train.append(train_data[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshape the input data for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Build the LSTM model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(window_size, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the LSTM model
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Prepare the test data
X_test = []
y_test = []
for i in range(window_size, len(test_data)):
    X_test.append(test_data[i - window_size:i, 0])
    y_test.append(test_data[i, 0])
X_test, y_test = np.array(X_test), np.array(y_test)

# Reshape the input data for LSTM
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Make predictions with the LSTM model
predicted_data = model.predict(X_test)
predicted_data = scaler.inverse_transform(predicted_data)

# Create a DataFrame for the predicted values
predicted_df = pd.DataFrame(predicted_data, index=df.index[train_size+window_size:], columns=['predicted'])

# Plot the actual and predicted values
plt.figure(figsize=(10, 6))
plt.plot(df.index, df[target_column], label='Actual')
plt.plot(predicted_df.index, predicted_df['predicted'], color='red', label='Predicted')
plt.xlabel(date_column)
plt.ylabel(target_column)
plt.title('LSTM Forecast')
plt.legend()
plt.show()


In [ ]:
# Calculate MAPE
def calculate_mape(actual, predicted):
    return np.mean(np.abs((actual - predicted) / actual)) * 100

# Convert the actual and predicted data to numpy arrays
actual_data = df[target_column].values[train_size+window_size:]
predicted_data = predicted_df['predicted'].values

# Calculate MAPE
mape = calculate_mape(actual_data, predicted_data)
print(f"MAPE using LSTM: {mape:.3f}")

# Calculate MAPE using Review Score
mape_score = calculate_mape(actual_data, predicted_data* (review_Score/5))
print(f"MAPE using LSTM with review score: {mape_score:.3f}")

# Calculate MAPE using Review Text
mape_NLP = calculate_mape(actual_data, predicted_data* (review_NLP/5))
print(f"MAPE using LSTM with review text: {mape_NLP:.3f}")

# Calculate MAPE using Review Score tuning
mape_scoreTune = calculate_mape(actual_data, predicted_data* (comp_rating/5))
print(f"MAPE using LSTM with review score tuning: {mape_scoreTune:.3f}")


In [ ]:
# Calculate MSLE
def calculate_msle(actual, predicted):
    return np.mean(np.square(np.log1p(actual) - np.log1p(predicted)))

# Calculate MSLE
msle = calculate_msle(actual_data, predicted_data)
print(f"MSLE using LSTM: {msle:.3f}")

msle_score = calculate_msle(actual_data, predicted_data * (review_Score/5))
print(f"MSLE using LSTM with review score: {msle_score:.3f}")

msle_NLP = calculate_msle(actual_data, predicted_data * (review_NLP/5))
print(f"MSLE using LSTM with review text: {msle_NLP:.3f}")

mape_scoreTune =calculate_msle(actual_data, predicted_data * (comp_rating/5))
print(f"MSLE using LSTM with review score tuning: {mape_scoreTune:.3f}")
